# Using Enron data-set, perform 3 analysis.

So Greg Whalley: "Traitor or Trader??"

Greg whalley was one of the central characters in the Enron drama.He was never called in the courtroom although he was on a list.Hence it was always a mystery whether Greg was involved in the Enron Scandal or not?

# Analysis - 1

ANDREW FASTOW, the CFO of Enron was one on the main culprits and we can analyze with whom he was in contact and to whom he has been sending emails and recieving them from.

In [18]:
from email.parser import Parser
import os,csv

rootdir='C:\\Users\\ruchi\\Downloads\\maildir'

with open('andrew_recieved.csv', 'a',encoding='utf-8') as andrew_recieved: #creating 2 csv's for storing data
    writer = csv.writer(andrew_recieved)
    writer.writerow(['To','From','Content'])
with open('andrew_sent.csv', 'a',encoding='utf-8') as andrew_sent:
    writer = csv.writer(andrew_sent)
    writer.writerow(['To','From','Content'])
andrew_recieved_count=0
andrew_sent_count=0
for subdir,dirs,files in os.walk(rootdir):                               #iterarting over all the sub folders and emails in them
    dir=os.path.basename(subdir)
    if dir=="inbox" or dir=="sent":
        for filename in files:
            filename = os.path.join(subdir, filename)
            with open(filename, 'r') as f:
                email = Parser().parsestr(f.read())
                
                emails_to_andrew=str(email['To'])                         #collecting data if the email is sent to Andrew
                if "andrew.fastow@enron.com" in emails_to_andrew:
                    with open('andrew_recieved.csv', 'a',encoding='utf-8') as andrew_recieved:
                        writer = csv.writer(andrew_recieved)
                        words=email.get_payload()
                        words=words.lower()
                        writer.writerow([str(email['To']),str(email['From']),words])
                        andrew_recieved_count+=1
                    
                emails_from_Andrew=str(email['From'])                     #collecting data if the email is recieved from Andrew
                if "andrew.fastow@enron.com" in emails_from_Andrew:
                    with open('andrew_sent.csv', 'a',encoding='utf-8') as andrew_sent:
                        writer = csv.writer(andrew_sent)
                        words=email.get_payload()
                        words=words.lower()
                        writer.writerow([str(email['To']),str(email['From']),words])
                        andrew_sent_count+=1
                    
print("The number of emails sent to Andrew Fastow is "+str(andrew_recieved_count))
print("The number of emails Andrew Fastow sent is "+str(andrew_sent_count))


The number of emails sent to Andrew Fastow is 159
The number of emails Andrew Fastow sent is 3


So the number of emails recieved by the CFO,Andrew Fastow is 159 and the number of emails sent by Andrew Fastow is 3.Now analyzing from which email id Andrew recieved the most number of emails and had sent most number of emails

In [20]:
from collections import Counter

most_emails_andrew_got_from=[]
most_emails_andrew_sent_to=[]
with open('andrew_recieved.csv','r') as andrew_r:                 #creating 2 csvs to store the details of senders & recievers
    rows = csv.reader(andrew_r)
    for row in rows:
        if row:
            most_emails_andrew_got_from.append(row[1])        
most_emails_got_from=most_emails_andrew_got_from[1:]

with open('andrew_sent.csv','r') as andrew_s:
    rows = csv.reader(andrew_s)
    for row in rows:
        if row:
            most_emails_andrew_sent_to.append(row[0])
most_emails_sent_to=most_emails_andrew_sent_to[1:]

counts_got_from = Counter(most_emails_got_from)                  #gettung frequency of emails
counts_sent_to = Counter(most_emails_sent_to)


print("top email senders to Andrew Fastow ")
print("------------------------------------------") 
print(counts_got_from.most_common(30))                           #printing emailids with max number of counts

print("top recipients of Andrew Fastow ")
print("------------------------------------------")
print(counts_sent_to.most_common(5))


top email senders to Andrew Fastow 
------------------------------------------
[('joannie.williamson@enron.com', 28), ('paula.rieker@enron.com', 12), ('mike.mcconnell@enron.com', 10), ('maureen.mcvicker@enron.com', 8), ('karen.myer@enron.com', 7), ('rosalee.fleming@enron.com', 5), ('gay.mayeux@enron.com', 5), ('rebecca.carter@enron.com', 4), ('l..wells@enron.com', 4), ('billy.dorsey@enron.com', 4), ('kathy.mayfield@enron.com', 4), ('steven.kean@enron.com', 4), ('billy.lemmons@enron.com', 3), ('j..kean@enron.com', 3), ('bobbie.power@enron.com', 3), ('lisa.costello@enron.com', 3), ('denne@enron.com', 3), ('enronsato@hotmail.com', 3), ('sally.beck@enron.com', 2), ('kenneth.lay@enron.com', 2), ('cassandra.schultz@enron.com', 2), ('karen.denne@enron.com', 2), ('kevin.hannon@enron.com', 2), ('b..sanders@enron.com', 2), ('vince.kaminski@enron.com', 2), ('dorsey@enron.com', 2), ('danz@enron.com', 2), ('ben.glisan@enron.com', 2), ('tim.despain@enron.com', 2), ('bryan.seyfried@enron.com', 2)]
to

It can be analyzed that Andrew Fastow dint sent out many emails but he did sent out 3 emails and 2 of them were for greg.whalley@enron.com.Hence I decided to analyze it further to findout major keywords and talked about things in these 2 emails

In [25]:
from nltk.corpus import stopwords
from collections import Counter
import nltk
import string

content_email_sent_by_Andrew=''
with open('andrew_sent.csv','r') as andrew_s:
    rows = csv.reader(andrew_s)
    for row in rows:
        if row:
            if row[0]=="greg.whalley@enron.com":
                content_email_sent_by_Andrew+=row[2]         #collecting all content from the emails sent out by Andrew
                
string_punctuation = string.punctuation
ignoreChar=['\r','\n','',' ',"'s"]
#nums=['0','1','2','3','4','5','6','7','8','9']
emails_data = nltk.word_tokenize(content_email_sent_by_Andrew)         #tokenizing content
words_only = [l.lower() for l in emails_data if l not in string_punctuation if l not in ignoreChar] #removing unwanted words
filtered_emails_data=[word for word in words_only if word not in stopwords.words('english')] #removing stopwords
wnl = nltk.WordNetLemmatizer()                                         #lemmatizing words
counts=Counter([wnl.lemmatize(data) for data in filtered_emails_data]) #recording the count of words
commn_words=[] 
freq=[]
for a in counts.most_common(30):                                   #printing words
    commn_words.append(a[0])
    freq.append(a[1])
    print(a)

('petrobras', 26)
('transaction', 16)
('asep', 12)
('--', 10)
('meeting', 10)
('november', 10)
('approval', 8)
('letter', 8)
('vagner', 8)
('victor', 8)
('ceg', 6)
('october', 4)
('state', 4)
('agenda', 4)
('required', 4)
('day', 4)
('14', 4)
('u', 4)
('sends', 4)
('governor', 4)
('issue', 4)
('directive', 4)
('wait', 4)
('close', 4)
('recommendation', 4)
('soon', 4)
('want', 4)
('target', 4)
('monday', 4)
('closing', 4)


In [ ]:
# Plotting a graph to better understand the data

In [27]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x=freq,
    y=commn_words,
    mode='markers',
    name='word frequency graph',
    marker=dict(
        color='rgba(156, 165, 196, 0.95)',
        line=dict(
            color='rgba(156, 165, 196, 1.0)',
            width=.5,
        ),
        symbol='circle',
        size=10,
    )
)

data = [trace0]
layout = go.Layout(
    title="frequency of top 30 taked about words from emails sent out by Andrew Fastow to Greg Whalley",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor='rgb(102, 102, 102)',
        titlefont=dict(
            color='rgb(204, 204, 204)'
        ),
        tickfont=dict(
            color='rgb(102, 102, 102)',
        ),
        autotick=False,
        dtick=1,
        ticks='outside',
        tickcolor='rgb(102, 102, 102)',
    ),
    margin=dict(
        l=150,
        r=70,
        b=50,
        t=80
    ),
    legend=dict(
        font=dict(
            size=10,
        ),
        yanchor='middle',
        xanchor='right',
    ),
    width=800,
    height=600,
    paper_bgcolor='rgb(254, 247, 234)',
    plot_bgcolor='rgb(254, 247, 234)',
    hovermode='closest',
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='word-freequency_graph')

It can be noticed that the Most talked about words in these emails are "PETROBRAS" "meeting" "MONDAY" "November" and According to the research it was found out that the Enron Corp. and Petrobras, Brazil's big, government-owned oil and gas business, have formed a partnership in the month of November on Monday.

# Analysis - 2

This span of this data is over the years and I am analzying in which month and year most number of emails has been sent and deleted

In [12]:
from email.parser import Parser
import os
import csv

rootdir='C:\\Users\\ruchi\\Downloads\\maildir'

with open('dates_feq_sent.csv', 'a',encoding='utf-8') as fr: #creating 2 csv's for storing data
    writer = csv.writer(fr)
    writer.writerow(['MonthYear','Month','Year','freq'])
    
with open('dates_feq_del.csv', 'a',encoding='utf-8') as fd: #creating 2 csv's for storing data
    writer = csv.writer(fd)
    writer.writerow(['MonthYear','Month','Year','freq'])
    
dates={}
dates_deleted={}
for subdir,dirs,files in os.walk(rootdir):                   #iterating over all the subfolders and folders and files
    dir=os.path.basename(subdir)
    if dir=="sent":
        for filename in files:
            filename = os.path.join(subdir, filename)
            with open(filename, 'r') as f:
                email = Parser().parsestr(f.read())
                d=email['Date']                             #extracting the date
                d=d[7:16].replace(" ","")
                if d not in dates:
                    dates[d]=1
                else:                                       #stroing the dates and its count in a dictionary
                    dates[d]+=1
                    
    if dir=="deleted_items":
        for filename in files:
            filename = os.path.join(subdir, filename)      #iterating over all the subfolders and folders and file
            with open(filename, 'r') as deleted_f:
                email = Parser().parsestr(deleted_f.read())
                d=email['Date']                            #extracting the date
                d=d[7:16].replace(" ","")
                if d not in dates_deleted:
                    dates_deleted[d]=1                     #stroing the dates and its count in a dictionary
                else:
                    dates_deleted[d]+=1

print("SENT")
print("*****************************************************************************")
for date in sorted(dates.items()):                         #printing out the data and storing it in a csv
    print(date)
    with open('dates_feq_sent.csv', 'a',encoding='utf-8') as fr: #creating 2 csv's for storing data
        writer = csv.writer(fr)
        writer.writerow([date[0],date[0][:3],date[0][3:],date[1]])

print("DELETED")
print("*****************************************************************************")
for d_date in sorted(dates_deleted.items()):              #printing out the data and storing it in a csv
    print(d_date)
    with open('dates_feq_del.csv', 'a',encoding='utf-8') as fd: #creating 2 csv's for storing data
        writer = csv.writer(fd)
        writer.writerow([d_date[0],d_date[0][:3],d_date[0][3:],d_date[1]])

SENT
*****************************************************************************
('Apr1999', 20)
('Apr2000', 1726)
('Apr2001', 4228)
('Aug1999', 373)
('Aug2000', 3351)
('Aug2001', 141)
('Dec1998', 49)
('Dec1999', 746)
('Dec2000', 4096)
('Feb1999', 34)
('Feb2000', 1235)
('Feb2001', 4311)
('Jan1999', 58)
('Jan2000', 1284)
('Jan2001', 4369)
('Jul1999', 304)
('Jul2000', 2572)
('Jul2001', 569)
('Jul2002', 21)
('Jun1999', 250)
('Jun2000', 2611)
('Jun2001', 945)
('Jun2002', 1)
('Mar1999', 40)
('Mar2000', 1571)
('Mar2001', 4140)
('May1999', 266)
('May2000', 2119)
('May2001', 2992)
('Nov1998', 23)
('Nov1999', 343)
('Nov2000', 4431)
('Nov2001', 7)
('Oct1998', 4)
('Oct1999', 411)
('Oct2000', 4399)
('Oct2001', 73)
('Sep1999', 442)
('Sep2000', 3561)
('Sep2001', 52)
DELETED
*****************************************************************************
('Apr1986', 1)
('Apr2001', 135)
('Apr2002', 270)
('Aug0001', 3)
('Aug2001', 920)
('Dec0001', 24)
('Dec2001', 2574)
('Dec2005', 1)
('Feb0002', 5)
('Fe

In [ ]:
# Plotting a graph to better understand the trend

In [13]:
import plotly.plotly as py
import plotly.graph_objs as go

x_data_sent_year=[]
x_data_sent_freq=[]

for date in sorted(dates.items()):
    x_data_sent_year.append(date[0])
for date in sorted(dates.items()):
    x_data_sent_freq.append(date[1])
    
# Create a trace
trace0 = go.Scatter(
    x = x_data_sent_year,
    y = x_data_sent_freq,
    name = 'feq of sent mails',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

data = [trace0]

# Edit the layout
layout = dict(title = 'Frequency of sent mails for all months and years',
              xaxis = dict(title = 'Month Year'),
              yaxis = dict(title = 'Frequency'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Frequency_sent_emails')

In [14]:
import plotly.plotly as py
import plotly.graph_objs as go

x_data_del_year=[]
x_data_del_freq=[]

for date_d in sorted(dates_deleted.items()):
    x_data_del_year.append(date_d[0])
for date_d in sorted(dates_deleted.items()):
    x_data_del_freq.append(date_d[1])
    
# Create a trace
trace0 = go.Scatter(
    x = x_data_del_year,
    y = x_data_del_freq,
    name = 'feq of deleted mails',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4)
)

data = [trace0]

# Edit the layout
layout = dict(title = 'Frequency of deleted mails for all months and years',
              xaxis = dict(title = 'Month Year'),
              yaxis = dict(title = 'Frequency'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Frequency_deleted_emails')

It can be seen that the maximum number of emails were deleted in OCT 2011 and from the records its visible that in October only, the first article appeared which revealed the details of Fastow's partnerships and shows the precarious nature of Enron's business and the SEC begins an informal probe of Enron.Hence, people at Enron carried out a massive operation of destroying tons of docuents including emails 

# Analysis - 3

Kenneth Lay, CEO of Enron was also one of the major culprit involved in the Enron Scandal.I have analyzed emails sent and recieved by Kenneth Lay to further analyze the same.

In [17]:
from email.parser import Parser
import os,csv

rootdir='C:\\Users\\ruchi\\Downloads\\maildir'

with open('kenneth_recieved.csv', 'a',encoding='utf-8') as kenneth_recieved: #creating 2 csv's for storing data
    writer = csv.writer(kenneth_recieved)
    writer.writerow(['To','From','Content'])
with open('kenneth_sent.csv', 'a',encoding='utf-8') as kenneth_sent:
    writer = csv.writer(kenneth_sent)
    writer.writerow(['To','From','Content'])
kenneth_recieved_count=0
kenneth_sent_count=0
for subdir,dirs,files in os.walk(rootdir):                               #iterarting over all the sub folders and emails in them
    dir=os.path.basename(subdir)
    if dir=="inbox" or dir=="sent":
        for filename in files:
            filename = os.path.join(subdir, filename)
            with open(filename, 'r') as f:
                email = Parser().parsestr(f.read())
                
                emails_to_kenneth=str(email['To'])                         #collecting data if the email is sent to Kenneth Lay
                if "kenneth.lay@enron.com" in emails_to_kenneth:
                    with open('kenneth_recieved.csv', 'a',encoding='utf-8') as kenneth_recieved:
                        writer = csv.writer(kenneth_recieved)
                        words=email.get_payload()
                        words=words.lower()
                        writer.writerow([str(email['To']),str(email['From']),words])
                        kenneth_recieved_count+=1
                    
                emails_from_kenneth=str(email['From'])                     #collecting data if the email is recieved from Kenneth lay
                if "kenneth.lay@enron.com" in emails_from_kenneth:
                    with open('kenneth_sent.csv', 'a',encoding='utf-8') as kenneth_sent:
                        writer = csv.writer(kenneth_sent)
                        words=email.get_payload()
                        words=words.lower()
                        writer.writerow([str(email['To']),str(email['From']),words])
                        kenneth_sent_count+=1
                    
print("The number of emails sent to Kenneth lay is "+str(kenneth_recieved_count))
print("The number of emails Kenneth lay sent is "+str(kenneth_sent_count))

The number of emails sent to Kenneth lay is 1204
The number of emails Kenneth lay sent is 7


In [36]:
from collections import Counter

import sys
import csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

most_emails_kenneth_got_from=[]
most_emails_kenneth_sent_to=[]
with open('kenneth_recieved.csv','r') as kenneth_r:                 #reading 2 csvs to store the details of senders & recievers
    rows = csv.reader(kenneth_r)
    for row in rows:
        if row:
            most_emails_kenneth_got_from.append(row[1])        
most_emails_got_from=most_emails_kenneth_got_from[1:]

with open('kenneth_sent.csv','r') as kenneth_s:
    rows = csv.reader(kenneth_s)
    for row in rows:
        if row:
            most_emails_kenneth_sent_to.append(row[0])
most_emails_sent_to=most_emails_kenneth_sent_to[1:]

counts_got_from1 = Counter(most_emails_got_from)                  #gettung frequency of emails
counts_sent_to1 = Counter(most_emails_sent_to)


print("top email senders to Kenneth lay ")
print("------------------------------------------") 
print(counts_got_from1.most_common(30))                           #printing emailids with max number of counts

print("top recipients of Kenneth lay ")
print("------------------------------------------")
print(counts_sent_to1.most_common(30))

top email senders to Kenneth lay 
------------------------------------------
[('kenneth.thibodeaux@enron.com', 96), ('joannie.williamson@enron.com', 46), ('karen.denne@enron.com', 32), ('rosalee.fleming@enron.com', 27), ('paula.rieker@enron.com', 20), ('steven.kean@enron.com', 20), ('svarga@kudlow.com', 20), ('shona.wilson@enron.com', 18), ('j..kean@enron.com', 15), ('simone.la@enron.com', 13), ('john.allison@enron.com', 13), ('enron_update@concureworkplace.com', 13), ('terrie.james@enron.com', 13), ('fraisy.george@enron.com', 12), ('mike.mcconnell@enron.com', 11), ('lynda.l.phinney@williams.com', 11), ('mailings@cnn.com', 11), ('mschopper@houston.org', 10), ('beau@layfam.com', 10), ('maureen.mcvicker@enron.com', 8), ('mark.koenig@enron.com', 8), ('pr <.palmer@enron.com>', 8), ('cindy.olson@enron.com', 8), ('liz.taylor@enron.com', 7), ('nshaw@usenergyservices.com', 7), ('david.delainey@enron.com', 6), ('jeff.donahue@enron.com', 6), ('greg.piper@enron.com', 6), ('la.rose@enron.com', 6),

In [40]:
from nltk.corpus import stopwords
from collections import Counter
import nltk
import string

content_email_sent_by_kenneth=''
with open('kenneth_sent.csv','r') as kenneth_s:
    rows = csv.reader(kenneth_s)
    for row in rows:
        if row:
            if "greg.whalley@enron.com" in row[0]:
                content_email_sent_by_kenneth+=row[2]         #collecting all content from the emails sent out by Kenneth
                
string_punctuation = string.punctuation
ignoreChar=['\r','\n','',' ',"'s"]
#nums=['0','1','2','3','4','5','6','7','8','9']
emails_data = nltk.word_tokenize(content_email_sent_by_kenneth)         #tokenizing content
words_only = [l.lower() for l in emails_data if l not in string_punctuation if l not in ignoreChar] #removing unwanted words
filtered_emails_data=[word for word in words_only if word not in stopwords.words('english')] #removing stopwords
wnl = nltk.WordNetLemmatizer()                                         #lemmatizing words
counts=Counter([wnl.lemmatize(data) for data in filtered_emails_data]) #recording the count of words
commn_words=[] 
freq=[]
for a in counts.most_common(30):                                   #printing words
    commn_words.append(a[0])
    freq.append(a[1])
    print(a)

('program', 13)
('associate/analyst', 8)
('enron', 8)
('one', 4)
('company', 4)
('--', 4)
('purpose', 3)
("''", 3)
('committee', 3)
('afl-cio', 3)
('ken', 2)
('lay', 2)
('department', 2)
('subject', 2)
('2001', 2)
('created', 2)
('important', 2)
('continue', 2)
('remains', 2)
('new', 2)
('talent', 2)
('direction', 2)
('many', 2)
('business', 2)
('graduate', 2)
('leader', 2)
('also', 2)
('message', 2)
('every', 2)
('earlier', 2)


In [41]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x=freq,
    y=commn_words,
    mode='markers',
    name='word frequency graph',
    marker=dict(
        color='rgba(156, 165, 196, 0.95)',
        line=dict(
            color='rgba(156, 165, 196, 1.0)',
            width=.5,
        ),
        symbol='circle',
        size=10,
    )
)

data = [trace0]
layout = go.Layout(
    title="frequency of top 30 taked about words from emails sent out by Kenneth lay to Greg Whalley",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor='rgb(102, 102, 102)',
        titlefont=dict(
            color='rgb(204, 204, 204)'
        ),
        tickfont=dict(
            color='rgb(102, 102, 102)',
        ),
        autotick=False,
        dtick=1,
        ticks='outside',
        tickcolor='rgb(102, 102, 102)',
    ),
    margin=dict(
        l=150,
        r=70,
        b=50,
        t=80
    ),
    legend=dict(
        font=dict(
            size=10,
        ),
        yanchor='middle',
        xanchor='right',
    ),
    width=800,
    height=600,
    paper_bgcolor='rgb(254, 247, 234)',
    plot_bgcolor='rgb(254, 247, 234)',
    hovermode='closest',
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='word-freequency_Kenneth_graph')

It can be analyzed that the most talked about words are Program, Associate/Analyst, Enron.

So Greg Walley: "Traitor or Trader??" While doing the analysis on emails sent by Andrew Fastow and Kenneth Lay, the maximum number of emails were sent to Greg Whalley and When those emails were analyzed nothing suspicious/fraudulent was found.